# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902057


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:19,  2.86s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:55,  2.04s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.77s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.29s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  5.01it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.46it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.11it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.37it/s]

equidad1                               0.083557
kayleebug2017                          0.000731
nickoftona                             0.000295
Betelgeuse12                           0.000481
not-logged-in-4c210c39c344b38ad15d    45.732450
SeaPaperclip                           0.001354
Sansy999                               0.000596
ElisabethB                             0.002715
Thedunkmasta                           0.001504
MeepTastic                             0.001419
AlexYoung35                            0.000293
clairedeu                              0.000224
peakscience7                           0.002727
not-logged-in-c6c7368ea3093fa77386     0.000163
acapirala                              0.000704
jjddmmrr                               0.022846
surfgimp                               0.001598
aisha3540                              0.000480
Lavadude                               0.016103
not-logged-in-6ea7dd94a335e5238780     0.000950
w7250369                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())